# Quarter Wave Resonator Class

In [4]:
%matplotlib
import numpy as np
from scipy import optimize
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from math import pi

class QuarterWaveRes:
    def __init__(self, freq, dB, phase):   # phase in degree
        self.freq = freq
        self.dB = dB
        self.phase = phase
        self.s21 = np.power(10, dB / 20) * np.exp(1j * phase / 180 * np.pi)  # get s21 in complex
        self.s21_norm = 0
        self.xc1, self.xc2, self.yc1, self.yc2, self.R1, self.R2 = 0,0,0,0,0,0
        self.p0, self.p1, self.p2 = 0, 0, 0  # points on 1/s21 fitted circle
        
    def _calc_R(self,x,y, xc, yc):
        """ calculate the distance of each 2D point from the center (xc, yc) """
        return np.sqrt((x-xc)**2 + (y-yc)**2)

    def _f(self, c, x, y):
        """
        Function for least-square fit.
        calculate the algebraic distance between the data points and the mean circle centered at c=(xc, yc) """
        Ri = self._calc_R(x, y, *c)
        return Ri - Ri.mean()

    def _leastsq_circle_fit(self,x,y):
    # coordinates of the barycenter
        x_m = np.mean(x)
        y_m = np.mean(y)
        center_estimate = x_m, y_m
        center, ier = optimize.leastsq(self._f, center_estimate, args=(x,y)) # find center and R
        
        xc, yc, = center
        Ri       = self._calc_R(x, y, *center)
        R        =  self._get_R(x,y,*center)
        residue   = np.sum((Ri - R)**2)
        return xc, yc, R, residue
    
    def _get_R(self, x, y, xc, yc):  # calculate radius
        Ri = self._calc_R(x, y, xc, yc)
        return  Ri.mean()
        
    def _g(self, c0):
        """
        Function to be minimized when normalizing S21. This function sets algorithm to find amp and phi.
        """
        amp, phi = c0
        Z = amp * np.exp(1j*phi) * self.s21
        Cx, Cy, R, residu = self._leastsq_circle_fit(np.real(Z), np.imag(Z))  # circle_fit to S21
        Cx2, Cy2, R2, residu2 = self._leastsq_circle_fit(np.real(1/Z), np.imag(1/Z))  # circle_fit to S21^-1
        
        a = np.sqrt((Cx - 1)**2 + Cy**2)   # radius of s21 circle
        b = np.sqrt((Cx2 - 1)**2 + Cy2**2)  # radius of s21^-1 circles
        c = np.sqrt((Cx-Cx2)**2+(Cy-Cy2)**2)  # distance between two centers.
        
        Cc = Cx2 + Cy2 * 1j  # center of S21^-1 circle
        z1 = 1/Z[0] - Cc 
        z2 = 1/Z[-1] - Cc
        z3 = 1 - Cc
        theta1 = np.angle(z3)- np.angle(z1)
        theta2 = np.angle(z3)- np.angle(z2)
        
        return np.abs((a + b - c)/c)**2 + np.abs((theta1+theta2)/theta1)**2 # works better than second one
       
        #return np.abs(a + b - c) #+ np.abs( np.abs(1/Z[0]-1) - np.abs(1/Z[-1]-1) )
        
    def _normalizeS21(self, s21):
        """
        Normalized S21 by finding A and phi, where Aexp(1j*phi)* S21 is normalized, i.e. |S21|->1 as f>>f0
        Do this by minimizing |c-a-b|, where a,b, c are the radius of s21 and s21^-1 circles, and the distances
        between the centers of both circles. s21 and s21^-1 circles approach (1,0) point. 
        Also, minimize the angle difference between z3-z1 and z2-z3 (defined in _g function above)
        """      
        
        phi_init = np.mean(np.angle(s21)) * (-1)
        amp_init = 1/np.abs(s21[0])
        print('amp_init={:g}, phi_init(deg)={:g}'.format(amp_init, phi_init*180/np.pi))
        
        res = optimize.minimize(self._g, (amp_init, phi_init) )
        
        return res.x[0], res.x[1]    
    
    def _get_f0_delf(self,xc, yc, R, phi0):
        center = xc + 1j * yc
        #p0 = center + R * np.exp(1j*phi0) # f0
        #p1 = center + R * np.exp(1j*(phi0 + np.pi/2)) #f1
        #p2 = center + R * np.exp(1j*(phi0 - np.pi/2)) #f2
        
        #f0 = self.freq[np.argmin(np.abs(p0 - 1/self.s21_norm))]
        #del_f = np.abs(self.freq[np.argmin(np.abs(p1 - 1/self.s21_norm))] - self.freq[np.argmin(np.abs(p2 - 1/self.s21_norm))])
        
        # get f0 and del_f by interpolation
        
        angle = np.angle(1/self.s21_norm - center)
        f_interp = interp1d(angle, self.freq)
        
        f0 = f_interp(phi0)
        f1 = f_interp(phi0 + np.pi/2)
        f2 = f_interp(phi0 - np.pi/2)
        del_f = np.abs(f1-f2)
        
        # points corresponding to f0, f1, and f2 on the 1/s21 fit circle
        self.p0 = center + R * np.exp(1j*phi0)
        self.p1 = center + R * np.exp(1j*(phi0 + np.pi/2))
        self.p2 = center + R * np.exp(1j*(phi0 - np.pi/2))
        
        return f0, del_f
        
    
    def geometric_fit(self):
        # normalize S21
        amp, phi = self._normalizeS21(self.s21)
        self.s21_norm = self.s21 * amp * np.exp(1j* phi) 
        print('amp={:g}, phi(deg)={:g}'.format(amp, phi*180/np.pi))
    
        # circle-fit to both S21_norm and S21_norm^-1
        self.xc1, self.yc1, self.R1, residue1 = self._leastsq_circle_fit(np.real(self.s21_norm), np.imag(self.s21_norm))
        self.xc2, self.yc2, self.R2, residue2 = self._leastsq_circle_fit(np.real(1/self.s21_norm), np.imag(1/self.s21_norm))
        
        # Caluclate Q and others.
        phi0 = np.angle((self.xc2-1)+1j*self.yc2)
        f0, del_f =  self._get_f0_delf(self.xc2, self.yc2, self.R2, phi0)        
        
        Qi = f0 / del_f
        Qc = Qi / (2 * self.R2)
        QL = Qi * Qc / (Qi+ Qc)
        
        # create fit curve for S21
        self.s21_fit = 1/ (1 + Qi/Qc*np.exp(1j*phi0)/(1+2j*Qi*(self.freq-f0)/f0))
        
        return Qi, Qc, QL, f0, self.R1, self.R2, phi0
    
       
    def plot_fit(self):
               
        self.fig = plt.figure(figsize=(15,12))
        self.axes1 = self.fig.add_subplot(3,2,1)
        self.axes2 = self.fig.add_subplot(3,2,3)
        self.axes3 = self.fig.add_subplot(3,2,5)
        self.axes4 = self.fig.add_subplot(1,2,2)
        
        self.axes1.plot(self.freq, np.abs(self.s21_norm), self.freq, np.abs(self.s21_fit))
        self.axes1.grid()
        self.axes1.set_ylabel('|S21|')
        self.axes1.ticklabel_format(useOffset=False)

        self.axes2.plot(self.freq, 20*np.log10(np.abs(self.s21_norm)), self.freq, 20*np.log10(np.abs(self.s21_fit)))
        self.axes2.grid()
        self.axes2.set_ylabel('S21 (dB)')
        self.axes2.ticklabel_format(useOffset=False)

        self.axes3.plot(self.freq, np.angle(self.s21_norm, deg=True), self.freq, np.angle(self.s21_fit, deg=True))
        self.axes3.grid()
        self.axes3.set_xlabel('Frequency (Hz)')
        self.axes3.set_ylabel('Phase (degree)')
        self.axes3.ticklabel_format(useOffset=False)
        
        theta_fit = np.linspace(-pi, pi, 360)
        x_fit1 = self.xc1 + self.R1*np.cos(theta_fit)
        y_fit1 = self.yc1 + self.R1*np.sin(theta_fit)
        x_fit2 = self.xc2 + self.R2*np.cos(theta_fit)
        y_fit2 = self.yc2 + self.R2*np.sin(theta_fit)
        
        self.axes4.plot(np.real(self.s21_norm), np.imag(self.s21_norm), 'c.', markersize=8)
        self.axes4.plot(np.real(1/self.s21_norm), np.imag(1/self.s21_norm), 'k.',markersize=8)
        self.axes4.plot(1,0,'ro')
        self.axes4.plot(x_fit1, y_fit1,'r-', x_fit2, y_fit2, 'r-')
        self.axes4.plot(np.real(self.s21_fit),np.imag(self.s21_fit), 'b-')
        self.axes4.plot(np.real(1/self.s21_fit),np.imag(1/self.s21_fit), 'b-')
        self.axes4.plot(self.xc1, self.yc1,'ko', self.xc2, self.yc2, 'ko')
        self.axes4.plot(np.real(self.p0),np.imag(self.p0),'ro',np.real(self.p1),np.imag(self.p1),'bo',np.real(self.p2),np.imag(self.p2),'bo')
        self.axes4.grid()
        self.axes4.set_xlabel('real($S_{21}$)')
        self.axes4.set_ylabel('imag($S_{21}$)')
        self.axes4.set_aspect('equal', adjustable='box')
        
    def plot_data(self):
        self.fig = plt.figure(figsize=(15,12))
        self.axes1 = self.fig.add_subplot(3,2,1)
        self.axes2 = self.fig.add_subplot(3,2,3)
        self.axes3 = self.fig.add_subplot(3,2,5)
        self.axes4 = self.fig.add_subplot(1,2,2)
        
        self.axes1.plot(self.freq, np.abs(self.s21))
        self.axes1.grid()
        self.axes1.set_ylabel('|S21|')
        self.axes1.ticklabel_format(useOffset=False)

        self.axes2.plot(self.freq, 20*np.log10(np.abs(self.s21)))
        self.axes2.grid()
        self.axes2.set_ylabel('S21 (dB)')
        self.axes2.ticklabel_format(useOffset=False)

        self.axes3.plot(self.freq, np.angle(self.s21, deg=True))
        self.axes3.grid()
        self.axes3.set_xlabel('Frequency (Hz)')
        self.axes3.set_ylabel('Phase (degree)')
        self.axes3.ticklabel_format(useOffset=False)

        self.axes4.plot(np.real(self.s21), np.imag(self.s21), 'c.', np.real(1/self.s21), np.imag(1/self.s21),'ko', markersize=3)
        self.axes4.plot(1,0,'ro')
        self.axes4.grid()
        self.axes4.set_xlabel('real($S_{21}$)')
        self.axes4.set_ylabel('imag($S_{21}$)')
        self.axes4.set_aspect('equal', adjustable='box')



Using matplotlib backend: Qt5Agg


# Fit data

In [5]:
%matplotlib

#basepath = r'Z:\User\Jaseung\programs\pythonCode\Resonator Analysis' 
#basepath=''
#filename = 'Niobium05122016_5.551_0.1_0_400_100.txt'
#filename = 'W14_S21vsF_fr6.427815_6.427965_Pr-80_V0_T0.055_R5_fixAtt 29_002118.txt'
#baddatafilename = 'Niobium04122016_5.549_6.0_-20.000_0.0000_065609 0-82.txt'
#okfilename = 'Niobium04122016_5.832_0.2_-20.000_0.0000_071303 0-152.txt'
#baddatafilename = 'Niobium04122016_5.832_6.0_-20.000_0.0000_071312 0-162.txt'
#okfilename='Niobium04122016_5.953_0.6_-20.000_0.0000_073021 0-232.txt'
#baddata filename='Niobium04122016_5.953_6.0_-20.000_0.0000_073029 0-242.txt'

#filename='Niobium04122016_6.052_0.0_-20.000_0.0000_073408 0-292.txt'
#baddatafilename='Niobium04122016_6.052_6.0_-20.000_0.0000_073416 0-302.txt'
#filename='Niobium04122016_6.264_0.1_-20.000_0.0000_075120 0-372.txt'
#filename='Niobium04122016_6.264_6.0_-20.000_0.0000_075129 0-382.txt'
#filename='Niobium04122016_6.441_0.0_-20.000_0.0000_075507 0-432.txt'
#filename='Niobium04122016_6.441_6.0_-20.000_0.0000_075516 0-442.txt'

#basepath = ''
#filename = 'Niobium05122016_5.551_0.1_0_400_100.txt'
#filename = 'Niobium05122016_5.551_6.0_0_100_100.txt'

basepath = './'
filename = 'W14_S21vsF_fr6.427815_6.427965_Pr-80_V0_T0.055_R5_fixAtt 29_002118.dat'

path = basepath + filename
data = np.loadtxt(path, delimiter='\t', skiprows=0)
#data = np.loadtxt(path, delimiter='\t', skiprows=10)

# Make an instance
res1 = QuarterWaveRes(data[:,0], data[:,1], data[:,2])
res1.plot_data()

# Do geometric fit
DoFit = True
if DoFit:
    fit_param = res1.geometric_fit()
    res1.plot_fit()
    print('Qi={:g}\n Qc={:g}\n QL={:g}\n f0={:g}\n R1={:g}\n R2={:g}\n phi0(deg)={:g}'.format(fit_param[0], fit_param[1], 
                                                                                    fit_param[2],fit_param[3],
                                                                                    fit_param[4], fit_param[5],
                                                                                    fit_param[6]*180/np.pi))


Using matplotlib backend: Qt5Agg
amp_init=19.4271, phi_init(deg)=-49.0026
amp=16.805, phi(deg)=-42.0435
Qi=274236
 Qc=221678
 QL=122586
 f0=6.42791e+09
 R1=0.340895
 R2=0.618547
 phi0(deg)=-45.3911


In [ ]:
phi0def base():
    basepath=r'Z:\User\Yebin\Project\Resonator Qi test\Samples\020218_W14_HF blow dry_with post descum\R5\data\low power'
    filename = 'W14_S21vsF_fr6.427815_6.427965_Pr-80_V0_T0.055_R5_fixAtt 29_002118.dat'
    path = basepath + '\\'+filename
    dataS = np.loadtxt(path, delimiter='\t', skiprows=0)
    filenameW = 'W14_S21vsF_fr6.42489_6.43089_Pr-80_V0_T0.06_R5_fixAtt 29_012055.dat'
    pathW = basepath + '\\'+filenameW
    dataW = np.loadtxt(pathW, delimiter='\t', skiprows=0)
    res1 = QuarterWaveRes(dataS[:,0], dataS[:,1], dataS[:,2])
    res1.plot_data()

    res2 = QuarterWaveRes(dataW[:,0], dataW[:,1], dataW[:,2])
    res2.plot_data()

    # fit baseline
    p = len(dataW[:,0])
    w = int(p/5)
    freqW,freqW1,freqW2 = [],dataW[0:w,0] , dataW[(p-w):p,0]
    freqW.extend(freqW1)
    freqW.extend(freqW2)
    magdataW,magdataW1,magdataW2 =  [],dataW[0:w,1], dataW[(p-w):p,1]
    magdataW.extend(magdataW1)
    magdataW.extend(magdataW2)
    phasedataW,phasedataW1,phasedataW2 =  [],dataW[0:w,2] , dataW[(p-w):p,2]
    phasedataW.extend(phasedataW1)
    phasedataW.extend(phasedataW2)
    magBasePara = np.polyfit(freqW,magdataW,2)
    phaseBasePara = np.polyfit(freqW,phasedataW,1)
    magBaseFit = np.polyval(magBasePara,dataS[:,0])
    phaseBaseFit = np.polyval(phaseBasePara,dataS[:,0])
    # base line subtraction
    magSub = dataS[:,1] - magBaseFit
    phaseSub = dataS[:,2] - phaseBaseFit
    res3 = QuarterWaveRes(data[:,0], magSub, phaseSub)
    res3.plot_data()
    print(magSub)
    print(phaseSub)

In [77]:
base()

C:\Anaconda3\lib\site-packages\matplotlib\pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


[-1.46413672 -1.36400311 -1.36398571 -1.49251244 -1.50789158 -1.27546734
 -1.57407313 -1.47900383 -1.59517875 -1.58118628 -1.55984714 -1.63134732
 -1.55586741 -1.63237503 -1.55987826 -1.58272772 -1.77686559 -1.88416778
 -1.88108889 -1.77143532 -1.50462866 -1.78601413 -2.16913032 -1.82095902
 -1.95822605 -2.22134819 -2.19069035 -2.22085983 -2.31663353 -2.14524575
 -2.43635449 -2.60841204 -2.74970812 -2.65604371 -2.58578873 -2.76730366
 -2.73342991 -3.10942558 -3.01564727 -2.79741328 -3.55655921 -3.53550426
 -3.41026692 -3.74078721 -3.57630361 -3.88252294 -4.14372988 -4.31846914
 -4.57439312 -4.54998922 -4.92537503 -4.89874247 -5.43833543 -5.7016776
 -5.4229585  -5.73070411 -6.18138284 -6.37483529 -6.73373466 -6.45758335
 -6.53374856 -7.07797689 -6.29162663 -6.0125223  -5.19415058 -5.27151868
 -4.34640981 -4.07843905 -3.21001851 -2.54747929 -2.13616768 -1.8486118
 -1.31722744 -0.75873629 -0.13419397 -0.32000186  0.07707623  0.203612
  0.38559595  0.89188068  0.61289169  0.94889708  0.936